# Mínimos cuadrados para predicciones de precios
Gabriel Quinche

***Importado de librerias y de datos***

In [24]:
import Pkg 
Pkg.add("CSV")
Pkg.add("DataFrames")
using CSV
using DataFrames

    Updating registry at `C:\Users\GABRI\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\GABRI\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\GABRI\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\GABRI\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\GABRI\.julia\environments\v1.7\Manifest.toml`


In [25]:
df = CSV.read("BTC-USD.csv",DataFrame)

,Date,Open,High,Low,Close,Adj Close,Volume
,Date,Float64,Float64,Float64,Float64,Float64,Int64
1,2021-03-15,59267.4,60541.0,55393.2,55907.2,55907.2,66419369890
2,2021-03-16,55840.8,56833.2,53555.0,56804.9,56804.9,59749798599
3,2021-03-17,56825.8,58969.8,54528.6,58870.9,58870.9,60258313191
4,2021-03-18,58893.1,60116.2,54253.6,57858.9,57858.9,55746041000
5,2021-03-19,57850.4,59498.4,56643.7,58346.7,58346.7,49063873786
6,2021-03-20,58332.3,60031.3,58213.3,58313.6,58313.6,50361731222
7,2021-03-21,58309.9,58767.9,56005.6,57523.4,57523.4,51943414539
8,2021-03-22,57517.9,58471.5,54288.2,54529.1,54529.1,56521454974
9,2021-03-23,54511.7,55985.4,53470.7,54738.9,54738.9,56435023914


Acá tenemos multitud de datos relacionados al precio del bitcoin principalmente los Open estan relacionados a los valores del bitcoin a cierta hora especifica del día, considerara la hora de apertura (donde por ejemplo es cuando otros activos tradicionales como las acciones o el oro pueden ser comprados de forma abierta en la bolsa), otros a resaltar pueden ser los valores Low y High, que serían simplemente los máximos y mínimos del día, finalmente el volumen sera en $\$$ la cantidad de dinero que fue tranzado con la compra y venta del activo

In [ ]:
n = 10 #número de datos a usar

Procedemos entonces a extraer los datos a usar y por predecir en el modelo, en este caso intentantomos inferir precios solo a travez del tiempo, pues para usar los mismos datos del bitcoin para predecir podriamos terminar tomandos datos demasiado correlacionados al mismo precio y que no tendria sentido en usar siendo que es la variable a predecir

In [ ]:


bs = df[end-n+1:end,"Open"]
# xhat = Matrix(reshape(xhat,:,1))

In [54]:
xs = [i for i in 1:n]

10-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [13]:

A = [xhat ones(10)]

10×2 Matrix{Float64}:
  1.0  1.0
  2.0  1.0
  3.0  1.0
  4.0  1.0
  5.0  1.0
  6.0  1.0
  7.0  1.0
  8.0  1.0
  9.0  1.0
 10.0  1.0

En primera instancia usamos las ecuaciones normales de Gauss teniendo en cuenta que derivamos en clase eran el minimizador del problema $||Ax-b||^2$


In [56]:
x=  transpose(A) * A \ transpose(A) * yhat 

2×1 Matrix{Float64}:
 1.9999999999999996
 4.440892098500626e-15


Aunque en el procedimiento de arriba Julia puede usar tecnicas inteligentes como una factorización de Cholesky, que más adelante veremos porque esta asegurada junto a la inversa de la matriz $A^tA$ siempre que tengamos al menos tanta cantidad de datos diferentes como parametros, este proceso aumenta la sensibilidad de la matriz a los errores llamada condición $K(A^tA) = K(A)^2 \gg K(A)$ 

***Factorización QR***
La factorización QR es mucho más estable númericamente que las tecnicas anteriormente discutidas, una algóritmo en cierta manera naive de obtenerla es con la ortonormalización de Grahm-Smith, en Julia es fácil de implementar

In [77]:
betterx= qr(A) \ bs 

2-element Vector{Float64}:
     2.903503769696284
 39113.30572946667

Esto es equivalente a lo siguiente debido a la heuristicas para solución de sistemas de Julia 

In [78]:
betterx = A \ bs

2-element Vector{Float64}:
     2.903503769697026
 39113.305729466665

Acá tendriamos el primer modelo lineal, diciendo que el valor base del bitcoin en el día cero (que en este caso sería el comienzo de nuestro muestreo) es aprox $\$39.000$, mientras que tendriamos un aumento de aproximadamente $\$3$ por día

***Modelo No Lineal***

Acá construimos un modelo algo ad hoc, en el que intentamos relacionar los ciclos naturales de los meses, días, y horas al modelo por tanto intentamos usar muchos más datos par ael modelo

In [129]:
hourScale = 2 * pi * 24
dayScale = 2 * pi 
weekScale = 2* pi / 7 
monthScale = 2 * pi / 30
YearScale = 2 * pi / (365)
tempScale =  [ hourScale ; dayScale ; weekScale ; monthScale ; YearScale] 

n = 366 

println(tempScale)


function myModel(scales,t,params)
    return sum((params)*(sin.(t*scales)))

    end

[150.79644737231007, 6.283185307179586, 0.8975979010256552, 0.20943951023931953, 0.01721420632103996]


myModel (generic function with 3 methods)

Ya que queremos que sea relacionado a temporalidades del año tenemos que hacer un ajuste no solo a la escala del tiempo de los senos sino a las fases luego miramos cual es el primer dato de nuestro csv

In [128]:
first(df)

,Date,Open,High,Low,Close,Adj Close,Volume
,Date,Float64,Float64,Float64,Float64,Float64,Int64
1,2021-03-15,59267.4,60541.0,55393.2,55907.2,55907.2,66419369890


Como el punto cero va a corresponder a el 15 de marzo de 2021 debemos hace un ajuste de 15 días para el mes y de 75 días para el año. Por casualidad ese día corresponde a un lunes un punto cero natural como ciclo de la semana 

In [131]:
tempPhase = [ 0 ; 0 ; 0 ; 15; 75]

5-element Vector{Int64}:
  0
  0
  0
 15
 75

Para facilitar la construcción de la matriz podemos hacer un producto externo, pues queremos por ejemplo 

$\sin(at) + \sin(bt)+\sin(ct)...$

$\sin(at_2)+\sin(bt_2)+\sin(ct_3)$

por lo que podemos construir primero la matriz y luego aplicarle seno a la misma, podemos ver un ejemplo primero

In [138]:
outer = Vector{Float64}(1:n) * transpose(π* ones(n))


366×366 Matrix{Float64}:
    3.14159     3.14159     3.14159  …     3.14159     3.14159     3.14159
    6.28319     6.28319     6.28319        6.28319     6.28319     6.28319
    9.42478     9.42478     9.42478        9.42478     9.42478     9.42478
   12.5664     12.5664     12.5664        12.5664     12.5664     12.5664
   15.708      15.708      15.708         15.708      15.708      15.708
   18.8496     18.8496     18.8496   …    18.8496     18.8496     18.8496
   21.9911     21.9911     21.9911        21.9911     21.9911     21.9911
   25.1327     25.1327     25.1327        25.1327     25.1327     25.1327
   28.2743     28.2743     28.2743        28.2743     28.2743     28.2743
   31.4159     31.4159     31.4159        31.4159     31.4159     31.4159
   34.5575     34.5575     34.5575   …    34.5575     34.5575     34.5575
   37.6991     37.6991     37.6991        37.6991     37.6991     37.6991
   40.8407     40.8407     40.8407        40.8407     40.8407     40.8407
    ⋮      

In [140]:
#real useful matriz
outer = Vector{Float64}(1:n) * transpose(tempScale)

366×5 Matrix{Float64}:
   150.796     6.28319    0.897598   0.20944   0.0172142
   301.593    12.5664     1.7952     0.418879  0.0344284
   452.389    18.8496     2.69279    0.628319  0.0516426
   603.186    25.1327     3.59039    0.837758  0.0688568
   753.982    31.4159     4.48799    1.0472    0.086071
   904.779    37.6991     5.38559    1.25664   0.103285
  1055.58     43.9823     6.28319    1.46608   0.120499
  1206.37     50.2655     7.18078    1.67552   0.137714
  1357.17     56.5487     8.07838    1.88496   0.154928
  1507.96     62.8319     8.97598    2.0944    0.172142
  1658.76     69.115      9.87358    2.30383   0.189356
  1809.56     75.3982    10.7712     2.51327   0.20657
  1960.35     81.6814    11.6688     2.72271   0.223785
     ⋮                                         
 53532.7    2230.53     318.647     74.351     6.11104
 53683.5    2236.81     319.545     74.5605    6.12826
 53834.3    2243.1      320.442     74.7699    6.14547
 53985.1    2249.38     321.34   

Pongamos a prueba de inmediato hacer el desfase a la matriz para que estuviera lista para aplicar $\sin$

In [142]:
outer =outer .+ transpose(tempPhase)

366×5 Matrix{Float64}:
   150.796     6.28319    0.897598  15.2094  75.0172
   301.593    12.5664     1.7952    15.4189  75.0344
   452.389    18.8496     2.69279   15.6283  75.0516
   603.186    25.1327     3.59039   15.8378  75.0689
   753.982    31.4159     4.48799   16.0472  75.0861
   904.779    37.6991     5.38559   16.2566  75.1033
  1055.58     43.9823     6.28319   16.4661  75.1205
  1206.37     50.2655     7.18078   16.6755  75.1377
  1357.17     56.5487     8.07838   16.885   75.1549
  1507.96     62.8319     8.97598   17.0944  75.1721
  1658.76     69.115      9.87358   17.3038  75.1894
  1809.56     75.3982    10.7712    17.5133  75.2066
  1960.35     81.6814    11.6688    17.7227  75.2238
     ⋮                                       
 53532.7    2230.53     318.647     89.351   81.111
 53683.5    2236.81     319.545     89.5605  81.1283
 53834.3    2243.1      320.442     89.7699  81.1455
 53985.1    2249.38     321.34      89.9793  81.1627
 54135.9    2255.66     322.238

In [143]:
#finalmente aplicamos seno y agregamos un factor lineal mx+b al modelo
A = [sin.(outer ) ones(n) Vector{Float64}(1:n)]

366×7 Matrix{Float64}:
 -5.8783e-15   -2.44929e-16   0.781831      0.478129   -0.371858  1.0    1.0
 -1.17566e-14  -4.89859e-16   0.974928      0.285075   -0.355824  1.0    2.0
 -1.76349e-14  -7.34788e-16   0.433884      0.0795606  -0.339684  1.0    3.0
 -2.35132e-14  -9.79717e-16  -0.433884     -0.129431   -0.323444  1.0    4.0
 -8.62349e-14  -1.22465e-15  -0.974928     -0.332765   -0.307108  1.0    5.0
 -3.52698e-14  -1.46958e-15  -0.781831     -0.521556   -0.290681  1.0    6.0
  1.56953e-14  -1.71451e-15  -2.44929e-16  -0.687553   -0.274168  1.0    7.0
 -4.70264e-14  -1.95943e-15   0.781831     -0.8235     -0.257573  1.0    8.0
 -1.09748e-13  -2.20436e-15   0.974928     -0.923456   -0.240903  1.0    9.0
 -1.7247e-13   -2.44929e-15   0.433884     -0.983053   -0.224161  1.0   10.0
 -7.81793e-15  -9.79965e-15  -0.433884     -0.999686   -0.207352  1.0   11.0
 -7.05397e-14  -2.93915e-15  -0.974928     -0.972627   -0.190482  1.0   12.0
 -1.33261e-13   3.92135e-15  -0.781831     -0.90306  

Finalmente buscamos la parametrización optima del modelo

In [144]:
x= A \ df[:,"Open"]

7-element Vector{Float64}:
      5.611421716698393e13
     -8.060651155119328e10
     34.3669550072057
   1058.0562813825484
 -12092.712425137377
  59926.968102755476
    -69.66858680310679

Esta solución nos implica como modelo

$5.6 \sin(x \frac{2*24*\pi}{1}) -8.06\sin(x\frac{2*\pi}{1}) +34\sin(x\frac{2*\pi}{7})+1058\sin(x\frac{2*\pi}{30})+12092\sin(x\frac{2*\pi}{365}) + 59926-69.6x $

In [161]:
 A * (x) #podemos ver los valores predichos

366-element Vector{Float64}:
 64886.495574589804
 64424.97486166964
 63923.76647926502
 63406.433360335686
 62901.96514505183
 62443.39385474767
 62028.1318597901
 61637.30188744917
 61263.39630571341
 60906.09989735253
 60594.989699547266
 60327.83253421295
 60130.19934829941
     ⋮
 42527.36879630755
 42378.77869352754
 42202.3480975957
 41979.07466498088
 41283.78423397518
 40933.07151699845
 40539.42792838617
 40133.252932278105
 39737.567460667684
 39356.97020602251
 38572.5905806392
 38190.29746496977

In [167]:
last(df[:,2],10)-last(A*(x),10) # vector de errores recientes

10-element Vector{Float64}:
 -2798.148878595697
 -3549.7699769808823
 -3223.8818899751786
 -2190.255110998449
  1434.6423846138277
  -693.2841822781047
  -943.102616667682
  -472.2436430225134
  -726.2741746391985
  1526.6830040302302

In [177]:
(last(df[:,2],10)-last(A*(x),10)) ./ (0.01*last(df[:,2],10))

10-element Vector{Float64}:
 -7.101143873129338
 -9.23714338784105
 -8.470546931089041
 -5.6533192838795525
  3.417925337990148
 -1.7578213275792838
 -2.4310236536580123
 -1.2144707826539474
 -1.9190088854302818
  3.84390501493899

Por lo que vemos que incluso con los valores ya dados tenemos errores dentro del $\pm 10\%$  esto debido a que tenemos demasiados datos para tan pocos parametros  que por ejemplo no dimos un peso especial a los valores, finalmente a parte de predecir con el modelo queremos acotar el error relativo del modelo esto lo podemos hacer obteniendo máximos y mínimos para la misma formula anterior

In [239]:
describe(DataFrame([ (df[:,2]-A*(x)) ./ (0.01*df[:,2])],:auto))

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,x1,-2.31456,-38.4873,-2.4712,29.6565,0,Float64


***Segundo punto***

Vamos a usar este mismo método para rápidamente establecer una función que no solo tenga en cuenta el tiempo sino los valores del oro la plata y el petroleo

Ya tenemos una matriz A que tiene en cuenta los valores del bitcoin agregaremos otros terminos lineales a nuestro modelo $gx+sy+pz$  encontraremos nuevos parametros optimos


In [334]:
dfGold =CSV.read("Gold.csv",DataFrame)[:,["Date","Open"]]

,Date,Open
,String15,Float64
1,"Mar 15, 2022",172.6
2,"Mar 14, 2022",179.65
3,"Mar 11, 2022",189.26
4,"Mar 10, 2022",190.33
5,"Mar 09, 2022",206.83
6,"Mar 08, 2022",223.69
7,"Mar 07, 2022",210.69
8,"Mar 04, 2022",195.25
9,"Mar 03, 2022",185.89


In [335]:
dfSilver = CSV.read("Silver.csv",DataFrame)[:,["Date","Open"]]

,Date,Open
,String15,Float64
1,"Mar 15, 2022",25.26
2,"Mar 14, 2022",26.28
3,"Mar 13, 2022",26.15
4,"Mar 11, 2022",26.26
5,"Mar 10, 2022",26.06
6,"Mar 09, 2022",26.93
7,"Mar 08, 2022",25.81
8,"Mar 07, 2022",26.03
9,"Mar 06, 2022",25.86


In [336]:
dfPetrol = CSV.read("Petrol.csv",DataFrame)[:,["Date","Open"]]

,Date,Open
,String15,Float64
1,"Mar 15, 2022",1953.7
2,"Mar 14, 2022",1988.7
3,"Mar 11, 2022",2000.3
4,"Mar 10, 2022",1992.8
5,"Mar 09, 2022",2060.2
6,"Mar 08, 2022",2001.0
7,"Mar 07, 2022",1978.5
8,"Mar 04, 2022",1938.5
9,"Mar 03, 2022",1931.7


Sin embargo si observamos con detenemiento los anteriores dataframes vemos que tenemos datos incompletos, si bien esto tal vez nos sugeriría buscar otro lugar de donde sacar los datos también es una buena oportunidad para poner en practica la limpieza de datos, parte muy requerida de cualquier análisis 

In [337]:
dfFutures = innerjoin(dfGold,dfSilver,dfPetrol,on=:Date,makeunique=true)

,Date,Open,Open_1,Open_2
,String15,Float64,Float64,Float64
1,"Mar 15, 2022",172.6,25.26,1953.7
2,"Mar 14, 2022",179.65,26.28,1988.7
3,"Mar 11, 2022",189.26,26.26,2000.3
4,"Mar 10, 2022",190.33,26.06,1992.8
5,"Mar 09, 2022",206.83,26.93,2060.2
6,"Mar 08, 2022",223.69,25.81,2001.0
7,"Mar 07, 2022",210.69,26.03,1978.5
8,"Mar 04, 2022",195.25,25.29,1938.5
9,"Mar 03, 2022",185.89,25.45,1931.7


Un problema no menor sin embargo es que estamos perdiendo las referencias temporales absolutas, debido a que las bolsas no abren todos los días o simplemente a que tenemos datos incompletos el indice del día 3 no corresponde con el día 3 desde el 15 de marzo, de hecho corresponde con un día ***4 días atras*** entonces debemos agregar un indexamiento que mantenga la escala temporal incluso con días perdidos en los datos e idealmente que siga la convención que el tiempo positivo es el sentido convencional, para esto mismo usaremos un nuevo compendio de datos esta vez que tenga la misma estructura de fechas (la cual era diferente en nuestro primer dataframe) y que ademas tenga valores para todos los días, asi teniendo un indexamiento correcto, luego usando la tecnica del inner join podemos obtener los indices en los cuales podremos entrenar

In [338]:
dfBitcoin = CSV.read("bitcoinDayAware.csv",DataFrame)[:,["Date","Open","day"]]

,Date,Open,day
,String15,Float64,Float64
1,"Mar 15, 2022",39717.9,366.0
2,"Mar 14, 2022",37777.1,365.0
3,"Mar 13, 2022",38816.3,364.0
4,"Mar 12, 2022",38737.0,363.0
5,"Mar 11, 2022",39458.5,362.0
6,"Mar 10, 2022",41964.0,361.0
7,"Mar 09, 2022",38757.0,360.0
8,"Mar 08, 2022",38039.0,359.0
9,"Mar 07, 2022",38437.0,358.0


***Datos completos***

Así Finalmente tenemos fechas donde tenemos los datos para todos los insumos



In [339]:
dfFinal = innerjoin(dfGold,dfSilver,dfPetrol,dfBitcoin,on=:Date,makeunique=true)

,Date,Open,Open_1,Open_2,Open_3,day
,String15,Float64,Float64,Float64,Float64,Float64
1,"Mar 15, 2022",172.6,25.26,1953.7,39717.9,366.0
2,"Mar 14, 2022",179.65,26.28,1988.7,37777.1,365.0
3,"Mar 11, 2022",189.26,26.26,2000.3,39458.5,362.0
4,"Mar 10, 2022",190.33,26.06,1992.8,41964.0,361.0
5,"Mar 09, 2022",206.83,26.93,2060.2,38757.0,360.0
6,"Mar 08, 2022",223.69,25.81,2001.0,38039.0,359.0
7,"Mar 07, 2022",210.69,26.03,1978.5,38437.0,358.0
8,"Mar 04, 2022",195.25,25.29,1938.5,42489.0,355.0
9,"Mar 03, 2022",185.89,25.45,1931.7,43952.0,354.0


finalmente nos contentaremos con un pequeño modelo lineal por ahora a que a priori realmente no conocemos muy buenos modelos para imponer, vamos a usar los vectores $G$ $S$ $P$ $T$ dentro de una matriz y usaremos mínimos cuadrados para obtener la relación con el valor Open_4 relacionado al bictoin

In [350]:
G,S,P,T=dfFinal.Open,dfFinal.Open_1,dfFinal.Open_2,dfFinal.day;

In [351]:
A = [ G S P T ] 

253×4 Matrix{Float64}:
 172.6   25.26  1953.7  366.0
 179.65  26.28  1988.7  365.0
 189.26  26.26  2000.3  362.0
 190.33  26.06  1992.8  361.0
 206.83  26.93  2060.2  360.0
 223.69  25.81  2001.0  359.0
 210.69  26.03  1978.5  358.0
 195.25  25.29  1938.5  355.0
 185.89  25.45  1931.7  354.0
 185.89  25.51  1945.3  353.0
 165.45  24.46  1908.4  352.0
 165.45  24.62  1921.0  351.0
 156.28  24.33  1906.5  348.0
   ⋮                    
  99.1   24.68  1700.0   16.0
  98.85  25.05  1732.8   15.0
  99.5   25.35  1732.5   12.0
  99.5   25.2   1745.1   11.0
  99.5   25.25  1742.4   10.0
  99.5   25.77  1738.0    9.0
  99.5   26.05  1746.5    8.0
  98.87  26.22  1752.4    5.0
 100.37  26.51  1744.7    4.0
 103.99  25.97  1737.8    3.0
 104.79  26.38  1748.9    2.0
 105.69  26.07  1737.5    1.0

In [352]:
b= dfFinal.Open_3

253-element Vector{Float64}:
 39717.9
 37777.1
 39458.5
 41964.0
 38757.0
 38039.0
 38437.0
 42489.0
 43952.0
 44442.0
 43202.0
 37734.0
 38380.0
     ⋮
 57672.3
 55773.0
 51363.0
 52347.2
 54331.0
 54074.0
 57368.0
 57640.0
 58870.0
 56924.0
 55644.0
 59005.0

Finalmente podemos obtener $x$ para construir nuestro sistema lineal

In [353]:
x = A\b

4-element Vector{Float64}:
    57.33821211404594
 -5279.342107857203
   103.19619924400774
   -93.16295515878083

El modelo de predición sería entonces $B \approx G\times57.33 - S\times 5279 + P \times 103 - t \times 93.16 $ 
Podriamos interpretarlo como que el bitcoin va devaluandose naturalmente, exceptuando con una subida en el valor del Oro que es un bien que la gente usa como protección a la inflación del Fiat
